#Articles

https://habr.com/ru/post/559110/

https://github.com/Daniil193/Simple_KG

https://github.com/siddhaling/Text-Summarization-with-Pronoun-Replacement

#DATA

https://www.kaggle.com/datasets/vfomenko/russian-news-2020?resource=download

https://www.kaggle.com/datasets/dhruvildave/wikibooks-dataset

In [ ]:
!pip install pandas
!pip install tqdm
!pip install stanza
!pip install nltk
!pip install numpy

In [3]:
import pandas as pd
from tqdm import tqdm                                                           # отображает процесс выполнения
import stanza                                                                   # Stanza is a Python natural language analysis package.
import nltk      
from nltk import tokenize                                                        
from nltk.tokenize import word_tokenize, sent_tokenize                          # разбитие на слова или предложение                                                                   
nltk.download('punkt')  
nltk.download('stopwords')                                                      # для великого и могучего
from nltk.corpus import stopwords                                                     
import string as st
import numpy as np

C:\Users\nekna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nekna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nekna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##Tokenization

In [4]:
import pandas as pd
import zipfile 

zip_archive = zipfile.ZipFile("./datatest_2.zip", "r")

data_1 = pd.read_csv(zip_archive.extract('news.csv', '.'))
#data_2 = pd.read_csv(zip_archive.extract('news.csv', '.'))

In [5]:
data_1 = data_1["text"]

In [6]:
data_1

0        В 1930-е годы Советский Союз охватила лихорадк...
1        Олимпийская чемпионка по фигурному катанию  Ал...
2        Российский врач-диетолог Римма Мойсенко объясн...
3        В 2019 году телеканал «Ю» запустил адаптацию з...
4        Актер  Михаил Ефремов  систематически употребл...
                               ...                        
21668    \n\n    \n    \n        \n        \n        \n...
21669    \n\n    \n    \n        \n        \n        \n...
21670    \n\n    \n    \n        \n        \n        \n...
21671    \n\n    \n    \n        \n        \n        \n...
21672    \n\n    \n    \n        \n        \n        \n...
Name: text, Length: 21673, dtype: object

In [7]:
for index in np.arange(len(data_1)):
  if data_1[index].startswith("\n"):
    data_1 = data_1.drop(labels=index, axis=0)
for index in np.arange(len(data_1)):
  if "РИА Новости" in sent_tokenize(data_1[index])[0] and len(sent_tokenize(data_1[index])[0]) < 50:
    data_1[index] = data_1[index].replace(sent_tokenize(data_1[index])[0], "")
for index in np.arange(len(data_1)):
  data_1[index] = data_1[index].strip()
data_1

0        В 1930-е годы Советский Союз охватила лихорадк...
1        Олимпийская чемпионка по фигурному катанию  Ал...
2        Российский врач-диетолог Римма Мойсенко объясн...
3        В 2019 году телеканал «Ю» запустил адаптацию з...
4        Актер  Михаил Ефремов  систематически употребл...
                               ...                        
20798    Интенсивность полетов у российских границ само...
20799    Певец  Валерий Сюткин  в интервью порталу  Nat...
20800    Владимир Путин  примет участие в церемонии отк...
20801    Концертный директор Сергей Лавров высказался в...
20802    23 декабря отмечается профессиональный  День Ф...
Name: text, Length: 20803, dtype: object

In [8]:
sentences_list = [sent for sent in data_1]

In [9]:
def tokenize_ru(file_text):
  tokens = word_tokenize(file_text)

  # let's delete punctuation symbols
  tokens = [i for i in tokens if (i not in st.punctuation)]

  # deleting stop_words
  stop_words = stopwords.words('russian')
  stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])
  tokens = [i for i in tokens if (i not in stop_words)]

  # cleaning words
  tokens = [i.replace("«", "").replace("»", "") for i in tokens]

  return tokens

In [10]:
sentences = []
sentences_done = []
for num, sentence in enumerate(sentences_list):
  sentences.append(sent_tokenize(sentence, 'russian'))
  for i, _string in enumerate(sentences[num]):
    sentences_done.append(tokenize_ru(_string))

In [ ]:
print(sentences[0:10])

##Template search

In [12]:
nlp = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma,ner,depparse') #use_gpu=True tokenize_pretokenized=True

2022-10-03 21:41:28 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-10-03 21:41:30 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2022-10-03 21:41:30 INFO: Use device: cpu
2022-10-03 21:41:30 INFO: Loading: tokenize
2022-10-03 21:41:30 INFO: Loading: pos
2022-10-03 21:41:30 INFO: Loading: lemma
2022-10-03 21:41:30 INFO: Loading: depparse
2022-10-03 21:41:30 INFO: Loading: ner
2022-10-03 21:41:31 INFO: Done loading processors!


In [13]:
%%time
num_of_news = 15000
triplets = []
for i in range(num_of_news):
    for s in tqdm(sentences[i]):
        doc = nlp(s)
        for sent in doc.sentences:
                entities = [ent.text for ent in sent.ents]
                res_d = dict()
                temp_d = dict()
                for word in sent.words:
                    temp_d[word.text] = {"head": sent.words[word.head-1].text, "dep": word.deprel, "id": word.id}
                for k in temp_d.keys():
                    nmod_1 = ""
                    nmod_2 = ""
                    if (temp_d[k]["dep"] in ["nsubj", "nsubj:pass"]) & (k in entities):
                        res_d[k] = {"head": temp_d[k]["head"]}
                        
                        for k_0 in temp_d.keys():
                            if (temp_d[k_0]["dep"] in ["obj", "obl"]) &\
                            (temp_d[k_0]["head"] == res_d[k]["head"]) &\
                                (temp_d[k_0]["id"] > temp_d[res_d[k]["head"]]["id"]):
                                res_d[k]["obj"] = k_0
                                break
                        
                        for k_1 in temp_d.keys():
                            if (temp_d[k_1]["head"] == res_d[k]["head"]) & (k_1 == "не"):
                                res_d[k]["head"] = "не "+ res_d[k]["head"]
                        
                        if "obj" in res_d[k].keys():
                            for k_4 in temp_d.keys():
                                if (temp_d[k_4]["dep"] =="nmod") &\
                                (temp_d[k_4]["head"] == res_d[k]["obj"]):
                                    nmod_1 = k_4
                                    break
                                    
                            for k_5 in temp_d.keys():
                                if (temp_d[k_5]["dep"] =="nummod") &\
                                (temp_d[k_5]["head"] == nmod_1):
                                    nmod_2 = k_5
                                    break
                            res_d[k]["obj"] = res_d[k]["obj"]+" "+nmod_2+" "+nmod_1

                if len(res_d) > 0:
                    triplets.append([s, res_d])

100%|██████████| 11/11 [00:01<00:00,  7.44it/s]

CPU times: total: 2d 11h 10min 9s
Wall time: 9h 58min 35s


In [14]:
triplets

[['Петров сам выбрал наиболее богатый углем участок, выделил в бригаду Стаханова двоих крепильщиков и двоих откатчиков угля и даже лично спустился в шахту, чтобы контролировать процесс работы и светить в забое.',
  {'Петров': {'head': 'выбрал', 'obj': 'участок  '}}],
 ['Внушительный рекорд удалось установить с первой же попытки: в ночь с 30 на 31 августа 1935 года Стаханов за смену в 5 часов 45 минут нарубил 102 тонны угля при норме в 7 тонн, таким образом перевыполнив ее почти в 15 раз. По правилам того времени, добытый уголь должны были разделить на всю бригаду, однако рекорд отдали единолично Стаханову.',
  {'Стаханов': {'head': 'нарубил', 'obj': 'тонны  угля'}}],
 ['В сообщении говорилось, что автором рекорда является «А.',
  {'А.': {'head': 'является'}}],
 ['Стаханов», и редакторы предположили, что «А.» означает «Алексей».',
  {'Стаханов': {'head': 'предположили'}}],
 ['Об ошибке стало известно почти сразу, однако менять имя было поздно: Стаханов уже стал всесоюзной знаменитостью 

In [15]:
clear_triplets = []
for tr in triplets:
    for k in tr[1].keys():
        if "obj" in tr[1][k].keys():
            clear_triplets.append([tr[0], k, tr[1][k]['head'], tr[1][k]['obj']])
clear_triplets

[['Петров сам выбрал наиболее богатый углем участок, выделил в бригаду Стаханова двоих крепильщиков и двоих откатчиков угля и даже лично спустился в шахту, чтобы контролировать процесс работы и светить в забое.',
  'Петров',
  'выбрал',
  'участок  '],
 ['Внушительный рекорд удалось установить с первой же попытки: в ночь с 30 на 31 августа 1935 года Стаханов за смену в 5 часов 45 минут нарубил 102 тонны угля при норме в 7 тонн, таким образом перевыполнив ее почти в 15 раз. По правилам того времени, добытый уголь должны были разделить на всю бригаду, однако рекорд отдали единолично Стаханову.',
  'Стаханов',
  'нарубил',
  'тонны  угля'],
 ['Об ошибке стало известно почти сразу, однако менять имя было поздно: Стаханов уже стал всесоюзной знаменитостью под именем Алексей и привлек внимание советских властей, в том числе лично товарища Сталина.',
  'Стаханов',
  'стал',
  'знаменитостью  именем'],
 ['Так, Стаханов за свою ударную смену с 30 на 31 августа  получил  премию в размере месячно

##Graph drawing

In [23]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize=(500, 500))#размер картинки

G = nx.DiGraph()
edge_labels = [clear_triplets[i][2] for i in np.arange(len(clear_triplets))]
for i in np.arange(len(clear_triplets)):
    G.add_edge( clear_triplets[i][1], clear_triplets[i][3], name = clear_triplets[i][2] )


pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels = True)
edge_labels = nx.get_edge_attributes(G,'name')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.savefig('Graph.png')
plt.show()